# 컴포지션과 상속
상속은 많이 사용하는 개념이지만, 부모 클래스를 확장하여 새로운 클래스를 만들 때마다 부모와 강력하게 결합된 새로운 클래스가 생기게 된다.  
-> 소프트웨어 설계 시 결합력을 최소로 하는 것이 중요하다!.  
상속에서 많이 사용하는 기능은 코드 재사용으로 부모 클래스에 있는 메서드를 공짜로 얻을 수 있는데 코드 재사용시 여러 상황에서 동작 가능하고 쉽게 조합할 수 있는 응집력 높은 객체를 사용해야 한다.  

## 상속이 좋은 선택인 이유

새로운 하위 클래스 생성 시 클래스가 올바르게 정의되었는지 확인하기 위해 상속된 모든 메서드를 실제로 사용할 것인지 생각해봐야 한다!!  
-> 대부분의 메서드를 필요로 하지 않고 재정의하거나 대체하거나 한다면 설계상의 실수라고 볼 수 있다.   
1. 상위 클래스는 잘 정의된 인터페이스 대신 막연한 정의와 너무 많은 책임을 가졌다.  
2. 하위 클래스는 확장하려고 하는 상위 클래스의 적절한 세분화가 아니다. 



### 상속 안티패턴

개발자가 코드 재사용만을 목적으로 상속을 사용하려고 할 때 자주 발생되는 문제  
-> 도메인 문제를 해결하기 위해 적절한 데이터 구조를 만든 다음에 이 데이터 구조를 사용하는 객체를 만들지 않고, 데이터 구조 자체를 객체로 만드는 경우.  

In [2]:
import collections

class TransactionalPolicy(collections.UserDict):
    def change_in_policy(self, customer_id, **new_policy_data):
        self[customer_id].update(**new_policy_data)


In [6]:
from datetime import datetime


policy = TransactionalPolicy({
    "client001": {
        "fee": 1000.0,
        "expiration_date": datetime(2020, 1, 3),
        }
        })


In [9]:
policy["client001"]

{'fee': 1000.0, 'expiration_date': datetime.datetime(2020, 1, 3, 0, 0)}

In [10]:
policy.change_in_policy("client001", expiration_date=datetime(2020, 1, 4))

In [12]:
policy["client001"]

{'fee': 1000.0, 'expiration_date': datetime.datetime(2020, 1, 4, 0, 0)}

In [13]:
dir(policy)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'change_in_policy',
 'clear',
 'copy',
 'data',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

위와 같은 코드 작성시 2가지 문제점이 있다.
1. 계층구조가 잘못된 것으로, 일반적으로 기본클래스에서 새 클래스를 만드는 것은 개념적으로 확장되고 세부적인 것이라는 것을 의미하는데 현 코드에서 TransactionalPolicy라는 이름을 보고 사전 타입이라는 것을 알기는 쉽지 않다.   
2. 결합력에 대한 문제로, 사전의 모든 메서드를 포함하는데 사용하지도 않는 메서드들도 있고, 사전 타입을 확장함으로써 얻는 이득도 별로 없다.  

--> 위 방법을 해결하기 위해 컴포지션을 사용하여, TransactionalPolicy 자체가 사전이 되는 것이 아니라 사전을 활용하는 것으로 사전을 Private 속성에 저장하고 __getitem__으로 사전의 Proxy를 만들고 나머지 필요한 Public 메서드를 추가적으로 구현하는 것.  

In [ ]:
class TransactionalPolicy:
    def __init__(self, policy_data, **extra_data):
        self._data = {**policy_data, **extra_data}

    def change_in_policy(self, customer_id, **new_policy_data):
        self._data[customer_id].update(**new_policy_data)
    
    def __getitem__(self, customer_id):
        return self._data[customer_id]

    def __len__(self):
        return len(self._data)



### 파이썬의 다중상속
파이썬은 다중 상속을 지원한다.  

#### 메서드 결정 순서(MRO)

2개 이상의 클래스를 확장하고, 해당 클래스들이 모두 하나의 같은 기본 클래스를 확장한 경우, 맨 아래 클래스가 최상위 클래스에서 오는 메서드를 해결하는 방법이 여러가지 있다.  

파이썬은 C3 Linearization 또는 MRO 알고리즘을 사용하여 문제를 해결한다.


In [14]:
class BaseModule:
    module_name = "top"

    def __init__(self, module_name):
        self.name = module_name

    def __str__(self):
        return f"{self.module_name}:{self.name}"


class BaseModule1(BaseModule):
    module_name = "module-1"


class BaseModule2(BaseModule):
    module_name = "module-2"


class BaseModule3(BaseModule):
    module_name = "module-3"


class ConcreteModuleA12(BaseModule1, BaseModule2):
    ''' 1, 2 '''


class ConcreteModuleB23(BaseModule2, BaseModule3):
    ''' 2, 3'''

In [18]:
str(ConcreteModuleA12("test"))

'module-1:test'

In [16]:
[cls.__name__ for cls in ConcreteModuleA12.mro()]

['ConcreteModuleA12', 'BaseModule1', 'BaseModule2', 'BaseModule', 'object']

#### 믹스인(Mixin)

믹스인은 코드를 재사용하기 위해 일반적인 행동을 캡슐화해놓은 기본 클래스이다.

In [19]:
class BaseTokenizer:
    def __init__(self, str_token):
        self.str_token = str_token

    def __iter__(self):
        yield from self.str_token.split('-')

In [20]:
tk = BaseTokenizer("28a2320b-fd3f-4627-asdfawe3fadf")
list(tk)

['28a2320b', 'fd3f', '4627', 'asdfawe3fadf']

In [22]:
class UpperIterableMixin:
    def __iter__(self):
        return map(str.upper, super().__iter__())


class Tokenizer(UpperIterableMixin, BaseTokenizer):
    pass

In [23]:
tk = Tokenizer("28a2320b-fd3f-4627-asdfawe3fadf")
list(tk)

['28A2320B', 'FD3F', '4627', 'ASDFAWE3FADF']

믹스인을 이용하기 때문에 새로운 코드가 필요 없어서 데코레이터 역할을 수행하며, 믹스인에서 __iter__를 호출하고, super() 호출을 통해 다음 클래스에 위임하여 원하는 결과를 얻을 수 있다.